# Undistort Dataset

In [17]:
class Args:
    uncalibrated_dataset_path = "../../resources/images/uncalibrated/calibration_dataset.npz"
    calibrated_dataset_path = "../../resources/images/calibrated/dataset_1.npz"
    calibration_file_path = "../../resources/calibrations/calibration.yml"
args = Args()

In [18]:
from models.dataset.dataset_interface import open_dataset, save_dataset
from models.calibration.calibration import StereoCalibration
from tqdm import tqdm

## Actual Undistortion of all images

In [19]:
# Open Dataset and calibration file
rs_rgb, rs_z, rs_cm, rs_dc, zv_rgb, zv_z, zv_cm, zv_dc = open_dataset(args.uncalibrated_dataset_path)
print(zv_cm)
print(zv_dc)
print(f"Opened dataset containing {len(rs_rgb)} image sets")

## Load Coefficients
calibration = StereoCalibration()
calibration.load_calibration(path=args.calibration_file_path)
print(f"Calibration loaded:\n{args.calibration_file_path}")

## Undistort Images
undistorted_rs_rgb = []
undistorted_rs_depth = []
undistorted_zv_rgb = []
undistorted_zv_depth = []
for rs_rgb_img, rs_depth_img, zv_rgb_img, zv_depth_img in tqdm(zip(rs_rgb, rs_z, zv_rgb, zv_z), desc='remap tuples'):
    rs_rgb_img, zv_rgb_img = calibration.remap(rs_rgb_img, zv_rgb_img)
    rs_depth_img, zv_depth_img = calibration.remap(rs_depth_img, zv_depth_img)
    undistorted_rs_rgb.append(rs_rgb_img)
    undistorted_rs_depth.append(rs_rgb_img)
    undistorted_zv_rgb.append(zv_rgb_img)
    undistorted_zv_depth.append(zv_depth_img)

print("Remapped all images")

## save image dataset
save_dataset(
    args.calibrated_dataset_path,
    undistorted_rs_rgb,
    undistorted_rs_depth,
    rs_cm,
    rs_dc,
    undistorted_zv_rgb,
    undistorted_zv_depth,
    zv_cm,
    zv_dc
)

print(f"saved undistorted images in {args.calibrated_dataset_path}")

Opened dataset containing 24 image sets
Calibration loaded:
../../resources/calibrations/calibration.yml


remap tuples: 24it [00:00, 93.57it/s]


Remapped all images
saved undistorted images in ../../resources/images/calibrated/dataset_1.npz
